<a href="https://colab.research.google.com/github/ytg000629/daegu_bigdata/blob/main/TEXTRANK%EB%AC%B8%EC%9E%A5%EC%9A%94%EC%95%BD_KEYBERT_GOOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji
!pip install konlpy
!pip install google-cloud-texttospeech
!pip install google-cloud-speech
!pip install pydub
!pip install keybert
!pip install pyLDAvis

In [2]:
dpath = "/content/drive/MyDrive/대구 빅데이터/output/"

In [3]:
import pandas as pd
import numpy as np
import emoji
import re
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import string
import networkx
import re
from konlpy.tag import Komoran
from google.cloud import texttospeech
from google.cloud import speech
import os
import io
import warnings
from konlpy.tag import Okt
warnings.filterwarnings("ignore")


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_csv(dpath+'대구 음식.csv')

In [6]:
data.head(20)

,date,title,link,content
0,2023-07-04 10:05:04,대구경북 물가상승률 21개월 만에 2%대 진입,https://n.news.naver.com/mnews/article/088/000...,[\n\n\n\n\n6월 대구경북 소비자물가지수 동북지방통계청 제공대구경북 소비자물...
1,2023-07-04 11:21:06,대구 6월 소비자물가 2.7% 상승…21개월 만에 2%대,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스] 최진석 기자 = 7일 서울 농협 하나로마트 양재...
2,2023-07-03 13:37:01,"잔반 없는 급식, 이 학교에선 무슨 음식을 먹길래 [ESG 세상]",https://n.news.naver.com/mnews/article/047/000...,"[\n[ESG 세상] 학교 급식 정책, 공급 중심에서 음식물쓰레기까지 눈을 돌리는 ..."
3,2023-07-03 19:56:04,"대구광역시 군위군, 편입 후 뭐가 바뀌나",https://n.news.naver.com/mnews/article/629/000...,[\n\n\n\n\n대구광역시 군위군청/군위군경북 군위군이 7월 1일 대구광역시 군...
4,2023-07-05 03:03:29,[시인 최원준의 음식문화 잡학사전] &lt;18&gt; 갯장어와 하모,https://n.news.naver.com/mnews/article/658/000...,[\n냄비 위 눈꽃처럼 피어 입 안서 사르르…日이 탐했던 그 맛- 일제강점기 생산량...
5,2023-06-28 10:31:04,"'유통기한 지난 음식보관' 식약처, 어린이집 점검...20곳 적발",https://n.news.naver.com/mnews/article/437/000...,[\n \n\n\n\n식품의약품안전처 자료 사진. 〈사진=연합뉴스〉 소비기...
6,2023-06-29 15:50:01,"""홍준표 대구시장 1년, 화려한 말과 초라한 빈손""",https://n.news.naver.com/mnews/article/657/000...,[\n\n\n\n\n더불어민주당 대구시당은 홍준표 대구시장의 취임 1년을 맞아 논평...
7,2023-06-29 16:09:01,"군위군 대구광역시 편입 D-2 ""미래 100년의 큰 역사, 대구광역시 군위군에서 ...",https://n.news.naver.com/mnews/article/088/000...,"[\n중학생은 대구 고등학교 진학 가능, 급행 시내버스, 출산축하금 등 다양한 혜택..."
8,2023-07-03 11:23:05,"대구테크노파크, 공공 배달앱 ‘대구로’ 신규 서비스 참가 기업 모집",https://n.news.naver.com/mnews/article/009/000...,[\n시민 생활 관련 전 분야 대상 이달 18일까지 아이디어 접수\n\n\n\n ...
9,2023-07-02 17:24:01,대구경북 미래 걸린 '운명의 7월'…특화단지 등 국가 공모사업 당락 판가름,https://n.news.naver.com/mnews/article/088/000...,[\n대구·경주 미래차 소부장 '추가 지정'…구미 포항 상주 국가첨단산업로봇테스트필...


In [7]:
for i in range(len(data)):
    data.content[i] = data.content[i].replace('\n','')
    data.content[i] = data.content[i].replace('\t','')
    data.content[i] = data.content[i].replace('■','')
    data.content[i] = data.content[i].replace('▲','')
    data.content[i] = data.content[i].replace('com(재)','')

In [8]:
data.head(20)

,date,title,link,content
0,2023-07-04 10:05:04,대구경북 물가상승률 21개월 만에 2%대 진입,https://n.news.naver.com/mnews/article/088/000...,[6월 대구경북 소비자물가지수 동북지방통계청 제공대구경북 소비자물가지수 상승률이 2...
1,2023-07-04 11:21:06,대구 6월 소비자물가 2.7% 상승…21개월 만에 2%대,https://n.news.naver.com/mnews/article/003/001...,[[서울=뉴시스] 최진석 기자 = 7일 서울 농협 하나로마트 양재점에 마늘이 진열돼...
2,2023-07-03 13:37:01,"잔반 없는 급식, 이 학교에선 무슨 음식을 먹길래 [ESG 세상]",https://n.news.naver.com/mnews/article/047/000...,"[[ESG 세상] 학교 급식 정책, 공급 중심에서 음식물쓰레기까지 눈을 돌리는 전과..."
3,2023-07-03 19:56:04,"대구광역시 군위군, 편입 후 뭐가 바뀌나",https://n.news.naver.com/mnews/article/629/000...,[대구광역시 군위군청/군위군경북 군위군이 7월 1일 대구광역시 군위군으로 편입됐다....
4,2023-07-05 03:03:29,[시인 최원준의 음식문화 잡학사전] &lt;18&gt; 갯장어와 하모,https://n.news.naver.com/mnews/article/658/000...,[냄비 위 눈꽃처럼 피어 입 안서 사르르…日이 탐했던 그 맛- 일제강점기 생산량 모...
5,2023-06-28 10:31:04,"'유통기한 지난 음식보관' 식약처, 어린이집 점검...20곳 적발",https://n.news.naver.com/mnews/article/437/000...,[ 식품의약품안전처 자료 사진. 〈사진=연합뉴스〉 소비기한이 지난 음식을 ...
6,2023-06-29 15:50:01,"""홍준표 대구시장 1년, 화려한 말과 초라한 빈손""",https://n.news.naver.com/mnews/article/657/000...,"[더불어민주당 대구시당은 홍준표 대구시장의 취임 1년을 맞아 논평을 내고 ""홍 시장..."
7,2023-06-29 16:09:01,"군위군 대구광역시 편입 D-2 ""미래 100년의 큰 역사, 대구광역시 군위군에서 ...",https://n.news.naver.com/mnews/article/088/000...,"[중학생은 대구 고등학교 진학 가능, 급행 시내버스, 출산축하금 등 다양한 혜택경북..."
8,2023-07-03 11:23:05,"대구테크노파크, 공공 배달앱 ‘대구로’ 신규 서비스 참가 기업 모집",https://n.news.naver.com/mnews/article/009/000...,[시민 생활 관련 전 분야 대상 이달 18일까지 아이디어 접수 대구테크노파크 전경...
9,2023-07-02 17:24:01,대구경북 미래 걸린 '운명의 7월'…특화단지 등 국가 공모사업 당락 판가름,https://n.news.naver.com/mnews/article/088/000...,[대구·경주 미래차 소부장 '추가 지정'…구미 포항 상주 국가첨단산업로봇테스트필드 ...


In [11]:
text = '''대구경북 소비자물가지수 상승률이 21개월 만에 2%대에 진입했다. 석유류 가격이 역대 최대폭으로 하락하면서 물가 상승 압박이 크게 떨어진 것으로 보인다.

4일 동북지방통계청이 발표한 '2023년 6월 대구경북 소비자물가동향'에 따르면 지난달 대구의 소비자물가지수는 전년 동월 대비 2.7% 상승한 것으로 나타났다. 상승 폭(전년 동기 대비)은 올해 1월 5.3%, 2월 4.8%, 3월 4.3%, 4월 3.8%, 5월 3.2%로 꾸준히 하락세를 보였다.

항목별 대구 소비자 물가는 주택·수도·전기·연료(7.0%), 음식·숙박(6.5%), 식료품·비주류음료(4.8%), 의류·신발(8.0%) 등은 오른 반면, 교통(-13.0%)은 내렸다.

같은기간 경북지역 소비자물가지수는 2.1% 상승했다. 경북 역시 상승 폭이 1월 5.3%, 2월 5%, 3월 4.1%, 4월 3.5%, 5월 3.1%로 하향 곡선을 그렸다.

경북의 항목별 소비자 물가는 주택·수도·전기·연료(6.7%), 음식·숙박(6.2%), 식료품·비주류음료(4.3%), 의류·신발(8.0%) 등은 오름세를 보였고 교통(-14.4%)은 하락했다.

2%대 물가상승률은 2021년 9월(대구 2.6%·경북 2.8%) 이후 21개월 만이다. 전국 물가상승률도 마찬가지로 2%대로 낮아졌다.

통계청은 석유류 가격이 1년 전에 비해 25.4% 떨어지면서 물가 상승이 둔화된 것으로 분석했다. 전체 물가상승률 중 석유류의 기여도는 -1.47%포인트에 달한다.

통계청 관계자는 "석유류 가격이 하락하고 서비스 부문의 상승률이 둔화하면서 전체 소비자 물가 상승률이 2021년 9월 이후 처음으로 2%대를 기록했다"고 설명했다
'''

In [15]:
import re
text = re.sub(r"\n+", " ", text)
sentences = re.split("[\.?!]\s+", text)

In [16]:
import pandas as pd

data = []
for sentence in sentences:
    if(sentence == "" or len(sentence) == 0):
        continue
    temp_dict = dict()
    temp_dict['sentence'] = sentence
    temp_dict['token_list'] = sentence.split() #가장 기초적인 띄어쓰기 단위로 나누자!

    data.append(temp_dict)

df = pd.DataFrame(data) #DataFrame에 넣어 깔끔하게 보기

In [17]:
df

,sentence,token_list
0,대구경북 소비자물가지수 상승률이 21개월 만에 2%대에 진입했다,"[대구경북, 소비자물가지수, 상승률이, 21개월, 만에, 2%대에, 진입했다]"
1,석유류 가격이 역대 최대폭으로 하락하면서 물가 상승 압박이 크게 떨어진 것으로 보인다,"[석유류, 가격이, 역대, 최대폭으로, 하락하면서, 물가, 상승, 압박이, 크게, ..."
2,4일 동북지방통계청이 발표한 '2023년 6월 대구경북 소비자물가동향'에 따르면 지...,"[4일, 동북지방통계청이, 발표한, '2023년, 6월, 대구경북, 소비자물가동향'..."
3,"상승 폭(전년 동기 대비)은 올해 1월 5.3%, 2월 4.8%, 3월 4.3%, ...","[상승, 폭(전년, 동기, 대비)은, 올해, 1월, 5.3%,, 2월, 4.8%,,..."
4,"항목별 대구 소비자 물가는 주택·수도·전기·연료(7.0%), 음식·숙박(6.5%),...","[항목별, 대구, 소비자, 물가는, 주택·수도·전기·연료(7.0%),, 음식·숙박(..."
5,같은기간 경북지역 소비자물가지수는 2.1% 상승했다,"[같은기간, 경북지역, 소비자물가지수는, 2.1%, 상승했다]"
6,"경북 역시 상승 폭이 1월 5.3%, 2월 5%, 3월 4.1%, 4월 3.5%, ...","[경북, 역시, 상승, 폭이, 1월, 5.3%,, 2월, 5%,, 3월, 4.1%,..."
7,"경북의 항목별 소비자 물가는 주택·수도·전기·연료(6.7%), 음식·숙박(6.2%)...","[경북의, 항목별, 소비자, 물가는, 주택·수도·전기·연료(6.7%),, 음식·숙박..."
8,2%대 물가상승률은 2021년 9월(대구 2.6%·경북 2.8%) 이후 21개월 만이다,"[2%대, 물가상승률은, 2021년, 9월(대구, 2.6%·경북, 2.8%), 이후..."
9,전국 물가상승률도 마찬가지로 2%대로 낮아졌다,"[전국, 물가상승률도, 마찬가지로, 2%대로, 낮아졌다]"


In [18]:
import math

In [19]:
# sentence similarity = len(intersection) / log(len(set_A)) + log(len(set_B))
similarity_matrix = []
for i, row_i in df.iterrows():
    i_row_vec = []
    for j, row_j in df.iterrows():
        if i == j:
            i_row_vec.append(0.0)
        else:
            intersection = len(set(row_i['token_list']) & set(row_j['token_list']))
            log_i = math.log(len(set(row_i['token_list'])))
            log_j = math.log(len(set(row_j['token_list'])))
            similarity = intersection / (log_i + log_j)
            i_row_vec.append(similarity)
    similarity_matrix.append(i_row_vec)

In [20]:
weightGraph = np.array(similarity_matrix)
weightGraph

array([[0.        , 0.        , 0.20677041, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.24136314, 0.        ,
        0.        , 0.        , 0.20448438],
       [0.        , 0.        , 0.18603688, 0.18603688, 0.        ,
        0.        , 0.18803637, 0.        , 0.        , 0.        ,
        0.79210179, 0.        , 0.36836852],
       [0.20677041, 0.18603688, 0.        , 0.        , 0.        ,
        0.22223162, 0.        , 0.        , 0.        , 0.        ,
        0.18330727, 0.        , 0.        ],
       [0.        , 0.18603688, 0.        , 0.        , 0.        ,
        0.        , 1.22300968, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.97467811, 0.        , 0.        ,
        0.        , 0.        , 0.18150835],
       [0.        , 0.        , 0.22223162, 0.        , 0.        ,
        0.        , 0.     

In [21]:
from sklearn.preprocessing import normalize

In [22]:
def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)
    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [23]:
R = pagerank(weightGraph) # pagerank를 돌려서 rank matrix 반환
R = R.sum(axis=1) # 반환된 matrix를 row 별로 sum
indexs = R.argsort()[-3:] # 해당 rank 값을 sort, 값이 높은 3개의 문장 index를 반환

In [24]:
#rank값이 높은 3개의 문장 출력
for index in sorted(indexs):
    print(df['sentence'][index])

4일 동북지방통계청이 발표한 '2023년 6월 대구경북 소비자물가동향'에 따르면 지난달 대구의 소비자물가지수는 전년 동월 대비 2.7% 상승한 것으로 나타났다
경북 역시 상승 폭이 1월 5.3%, 2월 5%, 3월 4.1%, 4월 3.5%, 5월 3.1%로 하향 곡선을 그렸다
통계청 관계자는 "석유류 가격이 하락하고 서비스 부문의 상승률이 둔화하면서 전체 소비자 물가 상승률이 2021년 9월 이후 처음으로 2%대를 기록했다"고 설명했다 


In [10]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [59]:
from konlpy.tag import Twitter
nlpy = Twitter()
keywords=[]
print('키워드 추출')
doc = text
n=3
for i in range(n):
    keywords_mmr = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,1),use_mmr = True,top_n = n, diversity = 0.2)
    keyword = keywords_mmr[i][0]
    nouns = nlpy.nouns(keyword)
    print(f'{i+1}.',nouns[0])
    #keywords.append(keyword)

print('')

키워드 추출
1. 경북
2. 둔화
3. 물가상승률

